In [2]:
!pip install mediapipe

## Static Image test

In [3]:
import cv2
import math

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2.imwrite('grayscale.jpg',img)

IMAGE_FILENAMES = ['thumbs_down.jpg']
# Preview the images.
images = {name: cv2.imread(f'./test_image/{name}') for name in IMAGE_FILENAMES}
for name, image in images.items():
  print(name)
  resize_and_show(image)

thumbs_down.jpg


In [4]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode


# STEP 2: Create an GestureRecognizer object.
base_options = BaseOptions(model_asset_path='gesture_recognizer.task')
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)


images = []
results = []
for image_file_name in IMAGE_FILENAMES:
  image_file_name = f'./test_image/{image_file_name}'
  # STEP 3: Load the input image.
  image = mp.Image.create_from_file(image_file_name)

  # STEP 4: Recognize gestures in the input image.
  recognition_result = recognizer.recognize(image)

  # STEP 5: Process the result. In this case, visualize it.
  gesture_recognition_result = recognizer.recognize(image)
  print(gesture_recognition_result.gestures[0][0])


Category(index=-1, score=0.7611576914787292, display_name='', category_name='Thumb_Down')


W20230510 11:44:25.783167 4052196160 gesture_recognizer_graph.cc:128] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
I20230510 11:44:25.783980 4052196160 hand_gesture_recognizer_graph.cc:249] Custom gesture classifier is not defined.


## Live video test

In [16]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
GestureRecognizerResult = mp.tasks.vision.GestureRecognizerResult
VisionRunningMode = mp.tasks.vision.RunningMode


# STEP 2: Create an GestureRecognizer object.


result_text = ""
def print_result(result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
    global result_text
    result_text = 'gesture recognition result: {}'.format(result)
    if len(result.gestures) != 0:
        result_text = result.gestures[0][0].category_name
    print(result_text)
    
base_options = BaseOptions(model_asset_path='gesture_recognizer.task')

options = GestureRecognizerOptions(
    base_options=base_options,
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)

recognizer = vision.GestureRecognizer.create_from_options(options)

cap_device =  0
cap_width = 1440
cap_height = 1080

cap = cv2.VideoCapture(cap_device)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_height)


pop = 0
while cap.isOpened():
    # Read from camera
    ret, frame = cap.read()
    if not ret:
        break
    
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    frame_timestamp_ms = pop #int(cap.get(cv2.CAP_PROP_POS_MSEC))
    pop += 1
    print(frame_timestamp_ms)
    recognition_result = recognizer.recognize_async(mp_image, frame_timestamp_ms)
    cv2.putText(frame, str(result_text), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
    cv2.imshow('Mediapipe Feed', frame)
    


    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

W20230517 11:48:23.133066 4052196160 gesture_recognizer_graph.cc:128] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
I20230517 11:48:23.133896 4052196160 hand_gesture_recognizer_graph.cc:249] Custom gesture classifier is not defined.


0
gesture recognition result: GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
1
gesture recognition result: GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
2
gesture recognition result: GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
3
gesture recognition result: GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
4
gesture recognition result: GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
5
gesture recognition result: GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
6
gesture recognition result: GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landmarks=[])
7
gesture recognition result: GestureRecognizerResult(gestures=[], handedness=[], hand_landmarks=[], hand_world_landma

KeyboardInterrupt: 

In [15]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode


# STEP 2: Create an GestureRecognizer object.
base_options = BaseOptions(model_asset_path='gesture_recognizer.task')
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)

pop = 0

cap_device =  0
cap_width = 1440
cap_height = 1080

cap = cv2.VideoCapture(cap_device)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_height)

while cap.isOpened():
  # Read from camera
  ret, frame = cap.read()
  if not ret:
      break
  pop += 1
   
  if pop % 30 == 0:
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    gesture_recognition_result = recognizer.recognize(mp_image)
    # print(gesture_recognition_result)
    if len(gesture_recognition_result.gestures) != 0:
        result_text = gesture_recognition_result.gestures[0][0]
        print(result_text)
    
  cv2.putText(frame, str(result_text), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
  cv2.imshow('Mediapipe Feed', frame)
  


  if cv2.waitKey(10) & 0xFF == ord('q'):
      break

cv2.destroyAllWindows()


W20230517 11:40:14.297416 4052196160 gesture_recognizer_graph.cc:128] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
I20230517 11:40:14.298434 4052196160 hand_gesture_recognizer_graph.cc:249] Custom gesture classifier is not defined.


Category(index=-1, score=0.716410219669342, display_name='', category_name='None')
Category(index=-1, score=0.6925832033157349, display_name='', category_name='Closed_Fist')
Category(index=-1, score=0.5889968276023865, display_name='', category_name='Closed_Fist')
Category(index=-1, score=0.7315627932548523, display_name='', category_name='Closed_Fist')
Category(index=-1, score=0.568245530128479, display_name='', category_name='None')
Category(index=-1, score=0.5721396803855896, display_name='', category_name='Open_Palm')
Category(index=-1, score=0.5302216410636902, display_name='', category_name='Open_Palm')
Category(index=-1, score=0.865069568157196, display_name='', category_name='Closed_Fist')


KeyboardInterrupt: 

In [1]:
import PySide6.QtCore

# Prints PySide6 version
print(PySide6.__version__)

# Prints the Qt version used to compile PySide6
print(PySide6.QtCore.__version__)

6.5.0
6.5.0
